# bing 扒下来的数据不靠谱，这个文件没用了

# 构造数据集

从bing上面爬取图片数据

In [ ]:
import os
from io import BytesIO

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
from PIL import Image

In [ ]:
api_url = u"https://cn.bing.com/images/async?q={keyword}&first={first}&count={count}"

In [ ]:
def get_raw_response(keyword,first=0,count=100):
    url = api_url.format(keyword=keyword,first=first,count=count)
    res = rq.get(url)
    if 200 == res.status_code:
        return res.text
    else:
        return None

In [ ]:
keyword_list = [u"卷云",u"卷积云",u"卷层云",u"高积云",u"高层云",u"雨层云",u"层积云",u"层云",u"积云",u"积雨云"]

In [ ]:
data_path = "data"
test_data_path = "test"

In [ ]:
#for kw in keyword_list:
#    os.system("mkdir %s"%os.path.join(data_path,kw))

In [ ]:
for kw in keyword_list:
    print("processing %s"%kw)
    res = get_raw_response(kw)
    if not res:
        continue
    soup = BeautifulSoup(res)
    tags = soup.findAll(lambda tag:tag.has_attr("class") and "mimg" in tag['class'])
    for i,t in enumerate(tags):
        if t.has_attr("src"):
            img_url = t["src"]
        elif t.has_attr("data-src"):
            img_url = t["data-src"]
        else:
            continue
        print("handling idx %d in %s"%(i,kw))
        raw_img = rq.get(img_url).content
        with BytesIO() as tmp:
            tmp.write(raw_img)
            img = Image.open(tmp)
            img_name = "%d.jpg"%i
            full_path = os.path.join(data_path,kw,img_name)
            img.save(full_path)